In [6]:
import pandas as pd
import os
import shutil

# Baca CSV
df = pd.read_csv('data.csv')

# Folder asal gambar
source_folder = 'images'

# Folder tujuan dataset (otomatis buat per kelas)
target_folder = 'dataset'
os.makedirs(target_folder, exist_ok=True)

for _, row in df.iterrows():
    img_name = row['image']
    label = row['name']

    class_folder = os.path.join(target_folder, label)
    os.makedirs(class_folder, exist_ok=True)

    src = os.path.join(source_folder, img_name)
    dst = os.path.join(class_folder, img_name)

    if os.path.exists(src):
        shutil.copyfile(src, dst)
    else:
        print(f"❌ File tidak ditemukan: {src}")


ModuleNotFoundError: No module named 'pandas'

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

image_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    'dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)
model.save('model.h5')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import io
import os

app = FastAPI()
model = load_model('model.h5')

# Ambil label langsung dari folder dataset
class_names = sorted(os.listdir('dataset'))

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = Image.open(io.BytesIO(contents)).convert('RGB')
    img = img.resize((128, 128))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = float(np.max(predictions))

    return JSONResponse({
        "predicted_class": predicted_class,
        "confidence": round(confidence, 4)
    })